In [32]:
import os

import numpy as np
import onnxruntime as ort
from keras_image_helper import create_preprocessor
from io import BytesIO
from urllib import request
from PIL import Image

In [33]:
model_name = os.getenv("MODEL_NAME", "hair_classifier_v1.onnx")

In [34]:
session = ort.InferenceSession(
    model_name, providers=["CPUExecutionProvider"]
)
inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

In [35]:
print(input_name)
print(output_name)

input
output


# Q2

In [36]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [37]:
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
img = download_image(url)

In [43]:
target_size = (200, 200)
prepare_image(img, target_size)

(1024, 1024)

# Q3

In [39]:
# Convert to numpy array
img_array = np.array(img)
first_pixel = img_array[0, 0]
print(f"First pixel RGB: {first_pixel}")

# Preprocessing: scale to [0,1] then normalize
first_pixel_normalized = first_pixel / 255.0

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

preprocessed = (first_pixel_normalized - mean) / std
print(f"R channel value: {preprocessed[0]:.4f}")

First pixel RGB: [ 57 100  18]
R channel value: -1.1418


# Q4

In [44]:
print(f"Original image size: {img.size}")

# Ensure the image is resized to exactly 200x200
img = img.resize((200, 200), Image.NEAREST)
print(f"After resize: {img.size}")

# Convert to numpy array with float32
img_array = np.array(img, dtype=np.float32)
print(f"Numpy array shape: {img_array.shape}")

# Preprocessing
# 1. Scale to [0, 1]
img_array = img_array / 255.0

# 2. Apply ImageNet normalization
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
img_array = (img_array - mean) / std

# 3. Transpose to (C, H, W) format and add batch dimension
img_array = img_array.transpose(2, 0, 1)  # (H, W, C) -> (C, H, W)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension -> (1, C, H, W)

print(f"Final array shape: {img_array.shape}, dtype: {img_array.dtype}")

# Load ONNX model
session = ort.InferenceSession('hair_classifier_v1.onnx')

# Get input name
input_name = session.get_inputs()[0].name

# Run inference
outputs = session.run(None, {input_name: img_array})

# The output is a logit, apply sigmoid to get probability
logit = outputs[0][0][0]
probability = 1 / (1 + np.exp(-logit))  # Sigmoid function

print(f"Model output (probability): {probability:.2f}")

Original image size: (1024, 1024)
After resize: (200, 200)
Numpy array shape: (200, 200, 3)
Final array shape: (1, 3, 200, 200), dtype: float32
Model output (probability): 0.52
